In [1]:
import json
import functools
import os
import re
from tqdm import tqdm
import hashlib
from datetime import datetime, timezone
import time
import pytz
from time import mktime
from datetime import datetime
import time
import pandas as pd

In [2]:
# Initialize data containers
src_data = {}
dst_data = {} 
event_data = [] 

# Load the JSON file
input_file = "CVE-2021-4034.json"  # Path to your file
with open(input_file, "r", encoding="utf-8") as f:
    data = json.load(f)

# Parse the data
for entry in tqdm(data):
    # Extract source (n)
    if "n" in entry:
        src = entry["n"]
        src_data[src["identity"]] = {
            "pid": src["properties"].get("pid"),
            "ppid": src["properties"].get("ppid"),
            "exe": src["properties"].get("exe"),
            "command_line": src["properties"].get("command line"),
            "cwd": src["properties"].get("cwd"),
            "hash": src["properties"].get("hash"),
            "name": src["properties"].get("name"),
        }
    
    # Extract destination (m)
    if "m" in entry:
        dst = entry["m"]
        dst_data[dst["identity"]] = {
            "path": dst["properties"].get("path"),
            "permissions": dst["properties"].get("permissions"),
            "hash": dst["properties"].get("hash"),
        }
    
    # Extract relationship properties
    if "relationship_properties" in entry:
        relationship = entry["relationship_properties"]
        event_data.append({
            "source": entry["n"]["identity"],  # Process
            "target": entry["m"]["identity"],  # File artifact
            "type": relationship.get("type"),
            "operation": relationship.get("operation"),
            "event_id": relationship.get("event id"),
            "time": relationship.get("time"),
        })


100%|██████████| 756/756 [00:00<00:00, 380750.94it/s]


In [11]:
df = pd.DataFrame(event_data)
print(f"Event Types: {df["operation"].unique()}")
df = pd.DataFrame(event_data)
print(f"Event Types Category: {df["type"].unique()}")

Event Types: ['mmap (read)' 'load' 'close' 'execve' 'open' 'fork' 'exit' 'update'
 'setuid' 'setgid']
Event Types Category: ['Used' 'WasTriggeredBy']


In [7]:
print(f"Total Nodes: {len(src_data.keys()) + len(dst_data.keys())}" )

Total Nodes: 192


In [ ]:
print(f"Total Events: {len(event_data) }" )

Total Nodes: 756
